# Optimization Algorithms in Python - Bedrooms problem

## Algorithms

### Importing the libraries

In [ ]:
import time
import random
import math
import sys

### Random search

In [ ]:
def random_search(domain, fitness_function):
  best_cost = sys.maxsize
  for i in range(1000):
    solution = [random.randint(domain[i][0],domain[i][1]) for i in range(len(domain))]
    cost = fitness_function(solution)
    if cost < best_cost:
      best_cost = cost
      best_solution = solution
  return best_solution

### Hill climb

In [ ]:
def hill_climb(domain, fitness_function, initial = []):
  count = 0
  
  if len(initial) > 0:
    solution = initial
  else:
    solution = [random.randint(domain[i][0],domain[i][1]) for i in range(len(domain))]
  
  while True:
    neighbors = []
    for i in range(len(domain)):
      if solution[i] > domain[i][0]:
        if solution[i] != domain[i][1]:
          neighbors.append(solution[0:i] + [solution[i] + 1] + solution[i + 1:])
      if solution[i] < domain[i][1]:
        if solution[i] != domain[i][0]:
          neighbors.append(solution[0:i] + [solution[i] - 1] + solution[i + 1:])

    actual = fitness_function(solution)
    best = actual
    for i in range(len(neighbors)):
      count += 1
      cost = fitness_function(neighbors[i])
      if cost < best:
        best = cost
        solution = neighbors[i]

    if best == actual:
      print('Count: ', count)
      break

  return solution

### Simulated annealing

In [ ]:
def simulated_anneling(domain, fitness_function, temperature = 50000.0,
                       cooling = 0.95, step = 1, initial = []):
  count = 0

  if len(initial) > 0:
    solution = initial
  else:
    solution = [random.randint(domain[i][0], domain[i][1]) for i in range(len(domain))]
   
  while temperature > 0.1:
    i = random.randint(0, len(domain) - 1)
    direction = random.randint(-step, step)
    temp_solution = solution[:] 
    temp_solution[i] += direction
    if temp_solution[i] < domain[i][0]:
      temp_solution[i] = domain[i][0]
    elif temp_solution[i] > domain[i][1]:
      temp_solution[i] = domain[i][1]

    count += 1
    cost_solution = fitness_function(solution)
    cost_solution_temp = fitness_function(temp_solution)
    probability = pow(math.e, (-cost_solution_temp - cost_solution) / temperature)

    if (cost_solution_temp < cost_solution or random.random() < probability):
      solution = temp_solution

    temperature = temperature * cooling

  print('Count: ', count)
  return solution

### Genetic algorithm

In [ ]:
def mutation(domain, step, solution):
  gene = random.randint(0, len(domain) - 1)
  mutant = solution
  if random.random() < 0.5:
    if solution[gene] != domain[gene][0]:
      mutant = solution[0:gene] + [solution[gene] - step] + solution[gene + 1:]
  else:
    if solution[gene] != domain[gene][1]:
      mutant = solution[0:gene] + [solution[gene] + step] + solution[gene + 1:]
  return mutant

def crossover(domain, solution1, solution2):
  gene = random.randint(1, len(domain) - 2)
  return solution1[0:gene] + solution2[gene:]

def genetic(domain, fitness_function, population_size = 100, step = 1,
            probability_mutation = 0.2, elitism = 0.2,
            number_generations = 500, search = False):
  population = []
  for i in range(population_size):
    if search == True:
      solution = random_search(domain, fitness_function)
    else:
      solution = [random.randint(domain[i][0], domain[i][1]) for i in range(len(domain))]
    
    population.append(solution)

  number_elitism = int(elitism * population_size)

  for i in range(number_generations):
    costs = [(fitness_function(individual), individual) for individual in population]
    costs.sort()
    ordered_individuals = [individual for (cost, individual) in costs]
    population = ordered_individuals[0:number_elitism]
    while len(population) < population_size:
      if random.random() < probability_mutation:
        m = random.randint(0, number_elitism)
        population.append(mutation(domain, step, ordered_individuals[m]))
      else:
        i1 = random.randint(0, number_elitism)
        i2 = random.randint(0, number_elitism)
        population.append(crossover(domain, ordered_individuals[i1], ordered_individuals[i2]))
  return costs[0][1]

## Bedroom problem

### Domain of the problem

In [ ]:
bedrooms = ['Lisbon', 'Madrid', 'London', 'Dublin', 'Paris']

In [ ]:
bedrooms[4]

'Paris'

In [ ]:
preferences = [('Mary', ('Lisbon', 'Paris')),
               ('Peter', ('Lisbon', 'Paris')),
               ('Stuart', ('Madrid', 'Lisbon')),
               ('Jessica', ('Madrid', 'Dublin')),
               ('Fred', ('Paris', 'Madrid')), 
               ('John', ('London', 'Madrid')), 
               ('Paul', ('London', 'Paris')), 
               ('Suzane', ('Dublin', 'London')), 
               ('Amanda', ('Dublin', 'London')), 
               ('Laura', ('Dublin', 'London'))]

In [ ]:
preferences[1]

('Peter', ('Lisbon', 'Paris'))

In [ ]:
preferences[1][0]

'Peter'

In [ ]:
preferences[1][1]

('Lisbon', 'Paris')

In [ ]:
preferences[1][1][0]

'Lisbon'

In [ ]:
preferences[1][1][1]

'Paris'

In [ ]:
# (0,9), (0,8), (0,7), (0,6), (0,5), (0,4), (0,3), (0,2), (0,1), (0,0)

In [ ]:
len(bedrooms) * 2

10

In [ ]:
domain = [(0, (len(bedrooms) * 2) - i - 1) for i in range(0, len(bedrooms) * 2)]
domain

[(0, 9),
 (0, 8),
 (0, 7),
 (0, 6),
 (0, 5),
 (0, 4),
 (0, 3),
 (0, 2),
 (0, 1),
 (0, 0)]

### Printing the solution

In [ ]:
# [6,1,2,1,2,0,2,2,0,0]

In [ ]:
len(bedrooms)

5

In [ ]:
# ['Lisbon', 'Madrid', 'London', 'Dublin', 'Paris']
#   0  1       2  3      4  5      6  7      8  9

In [ ]:
def print_solution(solution):
  vacancies = []
  for i in range(len(bedrooms)):
    vacancies += [i, i]
  #print(vacancies)

  for i in range(len(solution)):
    current = solution[i]
    bedroom = bedrooms[vacancies[current]]
    print(preferences[i][0], bedroom)
    del vacancies[current]

In [ ]:
print_solution([6,1,2,1,2,0,2,2,0,0])

Mary Dublin
Peter Lisbon
Stuart Madrid
Jessica Madrid
Fred London
John Lisbon
Paul Paris
Suzane Paris
Amanda London
Laura Dublin


### Fitness function

In [ ]:
preferences[0][1]

('Lisbon', 'Paris')

In [ ]:
# [6,1,2,1,2,0,2,2,0,0]
def fitness_function(solution):
  cost = 0
  vacancies = [0,0,1,1,2,2,3,3,4,4]
  for i in range(len(solution)):
    current = solution[i]
    bedroom = bedrooms[vacancies[current]]
    preference = preferences[i][1]
    if preference[0] == bedroom:
      cost += 0
    elif preference[1] == bedroom:
      cost += 1
    else:
      cost += 3

    del vacancies[current]
  
  return cost

In [ ]:
fitness_function([6,1,2,1,2,0,2,2,0,0])

14

### Aplications of optimization algorithms

In [ ]:
solution = random_search(domain, fitness_function)
cost = fitness_function(solution)
print(cost)
print_solution(solution)

6
Mary Lisbon
Peter Paris
Stuart Lisbon
Jessica Madrid
Fred Paris
John London
Paul Madrid
Suzane Dublin
Amanda London
Laura Dublin


In [ ]:
solution = hill_climb(domain, fitness_function)
cost = fitness_function(solution)
print(cost)
print_solution(solution)

Count:  52
16
Mary Paris
Peter London
Stuart Paris
Jessica Dublin
Fred Madrid
John Madrid
Paul London
Suzane Dublin
Amanda Lisbon
Laura Lisbon


In [ ]:
solution = simulated_anneling(domain, fitness_function)
cost = fitness_function(solution)
print(cost)
print_solution(solution)

Count:  256
18
Mary Paris
Peter Dublin
Stuart London
Jessica Lisbon
Fred Madrid
John Madrid
Paul London
Suzane Dublin
Amanda Paris
Laura Lisbon


In [ ]:
preferences

[('Mary', ('Lisbon', 'Paris')),
 ('Peter', ('Lisbon', 'Paris')),
 ('Stuart', ('Madrid', 'Lisbon')),
 ('Jessica', ('Madrid', 'Dublin')),
 ('Fred', ('Paris', 'Madrid')),
 ('John', ('London', 'Madrid')),
 ('Paul', ('London', 'Paris')),
 ('Suzane', ('Dublin', 'London')),
 ('Amanda', ('Dublin', 'London')),
 ('Laura', ('Dublin', 'London'))]

In [ ]:
solution = genetic(domain, fitness_function)
cost = fitness_function(solution)
print(cost)
print_solution(solution)

2
Mary Lisbon
Peter Lisbon
Stuart Madrid
Jessica Madrid
Fred Paris
John London
Paul Paris
Suzane London
Amanda Dublin
Laura Dublin
